In [2]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/collab_sandbox"
%cd drive/MyDrive/collab_sandbox/NER/ner_tenses_recognition/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/collab_sandbox/NER/ner_tenses_recognition
 1000_5_1_20_meaningful_classification.csv
 active_learning
 advanced_tutorial_crf_lstm.ipynb
 bestsellers.csv
 best-val-model-86.pt
 datasets
 index.html
 logs
 meaningful_classif
 meaningful_classif.ipynb
 med_5_2_21_medium_sentences_dataset.csv
 med_5_2_21_medium_sentences_dataset_labeled.csv
'med_5_2_21_medium_sentences_dataset_labeled__Sun May  2 19:36:07 2021_1619984167.8032556.csv'
 med_gen_4_23_21_medium_sentences_dataset.csv
 med_gen_4_23_21_medium_sentences_dataset_labeled.csv
 medium_articles
 medium_sentences_dataset-2-22-21.csv
 medium_sentences_dataset-2-22-21.gsheet
 medium_sentences_dataset_labeled.csv
 medium_sentences_dataset_labeled.gsheet
 medium_sentenses_labeled_all__4_23_21_80479.csv
'medium_sentenses_labeled_all__4_23_21_80479__Sun May  2 01:18:31 2021_1619918311.231216.c

In [3]:
!pip install -q 'transformers<=3.5.1' 'torch<=1.7.1' seqeval # pytorch-crf 
# !pip install datasets

     |████████████████████████████████| 1.3MB 4.0MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 2.9MB 44.5MB/s 
     |████████████████████████████████| 1.1MB 38.6MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [4]:
import torch
import sklearn
import warnings
import transformers
# import sklearn.preprocessing
import numpy as np
from sklearn import preprocessing
from transformers import AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
import random
from tqdm.notebook import tqdm
import torch.nn as nn
from transformers import AutoConfig

import pandas as pd
import os
import sys
from transformers import AutoModel, AutoTokenizer, AutoConfig
from typing import List
from typing import List
from sklearn.metrics import precision_recall_fscore_support
import warnings
from tqdm import tqdm_notebook

from tqdm.notebook import tqdm
import multiprocessing
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import freeze_support
from torch.utils.data import Dataset

import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
import spacy
from spacy import displacy

from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

class NERDataset():
    def __init__(self, 
                sentences, 
                tags, 
                transformer_tokenizer, 
                transformer_config, 
                TAGS_DICT = None
            ):
        self.sentences = sentences
        self.tags = tags
        self.transformer_tokenizer = transformer_tokenizer
        self.max_len = 128
        self.pad_token_id = transformer_config.pad_token_id
        self.tag_outside_transformed = 0 # let O tag always be zero
        self.TAGS_DICT = TAGS_DICT
    
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, item):
        sentence = self.sentences[item]
        tags = self.tags[item]
 
        tags = np.array([self.TAGS_DICT[tok] for tok in tags])
        
        assert len(sentence) == len(tags)

        input_ids = []
        target_tags = []
        tokens = []
        offsets = []
        
        for i, word in enumerate(sentence):
            wordpieces = self.transformer_tokenizer.tokenize(word)
            tokens.extend(wordpieces)
            offsets.extend([1]+[0]*(len(wordpieces)-1))
            target_tags.extend([tags[i]] * len(wordpieces)) 
        
        
        tokens = tokens[:self.max_len - 2] 
        target_tags = target_tags[:self.max_len - 2]
        offsets = offsets[:self.max_len - 2]

        input_ids = self.transformer_tokenizer.encode(tokens)
          
        target_tags = [self.tag_outside_transformed] + target_tags + [self.tag_outside_transformed] 
        masks = [1] * len(input_ids)
        
        token_type_ids = [0] * len(input_ids) 
        offsets = [1] + offsets + [1]

        padding_len = self.max_len - len(input_ids)
        input_ids = input_ids + ([self.pad_token_id] * padding_len)
        masks = masks + ([0] * padding_len)  
        offsets = offsets + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tags = target_tags + ([self.tag_outside_transformed] * padding_len)  

        return {'input_ids' : torch.tensor(input_ids[:self.max_len], dtype = torch.long,),
                'masks' : torch.tensor(masks[:self.max_len], dtype = torch.long),
                'token_type_ids' : torch.tensor(token_type_ids[:self.max_len], dtype = torch.long),
                'target_tags' : torch.tensor(target_tags[:self.max_len], dtype = torch.long),
                'offsets': torch.tensor(offsets[:self.max_len], dtype = torch.long)} 
      

def flatten(l):
    return [item for sublist in l for item in sublist]

def compute_f1_scores(y_pred, y_true, labels, average=None): 
    # check inputs.
    assert sum([len(t) < len(p) for t, p in zip(y_true, y_pred)]) == 0, "predictions too long"

    # assert sum([len(t) > len(p) for t, p in zip(y_true, y_pred)]) == 0

    y_true = [t[:len(p)] for t, p in zip(y_true, y_pred)]
    
    y_pred = flatten(y_pred)
    y_true = flatten(y_true) 
    labels = list(set(y_true))
    # print(y_true)
    # print(y_pred)
    labels.remove("O")
    print(labels)
    labels = np.array(labels)

    f1_scores = precision_recall_fscore_support(y_true = y_true,
                                                y_pred = y_pred,
                                                labels = labels,
                                                average=average) 

    return f1_scores




class NERModel(nn.Module):
    def __init__(self, transformer, device, n_tags, dropout = 0.1):
        super(NERModel, self).__init__()
        transformer_name = transformer.name_or_path
        transformer_config = AutoConfig.from_pretrained(transformer_name)
        self.transformer = transformer
        self.dropout = nn.Dropout(dropout)
        # self.pretags = nn.Linear(transformer_config.hidden_size, transformer_config.hidden_size)
        self.tags = nn.Linear(transformer_config.hidden_size, n_tags)
        self.device = device
        # amount_params = len([item for item in self.transformer.parameters()])
        # step = 0
        # freeze_params = int(amount_params * 0.7)
        # for param in self.transformer.parameters():
        #   if step < freeze_params:
        #     param.requires_grad = False
        #   step += 1

    def forward(self, 
                input_ids: torch.Tensor, 
                masks: torch.Tensor, 
                token_type_ids: torch.Tensor, 
                target_tags: torch.Tensor, 
                offsets: torch.Tensor 
                ):
        transformer_inputs = {
            'input_ids': input_ids.to(self.device),
            'attention_mask': masks.to(self.device),
            }
        outputs = self.transformer(**transformer_inputs)[0]
        outputs = self.dropout(outputs)
        outputs = self.tags(outputs)
        return outputs




class NERExperiment:
    def __init__(self, 
                 model_name = 'roberta-base',
                 TAGS = None, 
                 dataset_training = None,
                 dataset_validation = None,
                 max_len = 128,
                 network = NERModel,
                 train_batch_size = 4,
                 dropout = 0.1,
                 grad_clip = 1,
                 optimizer_class = AdamW,
                 ):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.model_name = model_name  
        self.dataset_training = dataset_training
        self.dataset_validation = dataset_validation
        self.tag_outside = TAGS['O']
        self.TAGS = TAGS
        self.TAGS_NUM = dict([(value, key) for key, value in TAGS.items()])
        self.max_len = max_len
        self.transformer_model = AutoModel.from_pretrained(model_name)
        self.transformer_tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                                                   do_lower_case=True
                                                                   )
        self.transformer_config = AutoConfig.from_pretrained(model_name)  
        self.network = network(self.transformer_model, self.device, len(TAGS), dropout = dropout)
        self.network.to(self.device)
        self.train_losses = []
        self.valid_loss = np.nan
        self.train_batch_size = train_batch_size
        self.dl_train = None
        self.dl_validate = None
        self.scheduler = None
        self.scheduler_class = get_cosine_schedule_with_warmup
        self.n_tags = len(TAGS)
        self.optimizer = None
        self.criterion = torch.nn.CrossEntropyLoss()
        self.clip = grad_clip
        self.optimizer_class = optimizer_class
        

    def experiment(self, 
                    epochs = 10,
                    warmup_steps = 300,
                    learning_rate = 5e-5,
                   ):
        self.setup()
        optimizer_parameters = self.network.parameters()

        num_train_steps = int(len(self.dataset_training.get('sentences')) / self.train_batch_size * epochs)
        
        self.optimizer = self.optimizer_class(optimizer_parameters, lr = learning_rate)
        self.scheduler = self.scheduler_class(
            self.optimizer, num_warmup_steps = warmup_steps, num_training_steps = num_train_steps
        )

        train_losses = []
        best_valid_loss = 0.268
        best_f1_score = 0.6
        best_parameters = None

        for epoch in range(epochs):
            
            print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

            train_loss = self.train()
            train_losses.append(train_loss)
            valid_loss = self.validate()
            
            train_f1 = f1_score(self.dataset_training.get('tags'), self.predict(self.dataset_training.get('sentences')))
            cur_f1_score = f1_score(self.dataset_validation.get('tags'), self.predict(self.dataset_validation.get('sentences')))
            print(f"Train Loss = {train_loss} Valid Loss = {valid_loss} train_score = {train_f1} val_score = {cur_f1_score}")
            # print(classification_report(self.dataset_training.get('tags'), self.predict(self.dataset_training.get('sentences'))))
            # print(classification_report(self.dataset_validation.get('tags'), self.predict(self.dataset_validation.get('sentences'))))
            # if valid_loss < best_valid_loss:
            #     best_parameters = self.network.state_dict()            
            #     best_valid_loss = valid_loss
            if best_f1_score < cur_f1_score:
              best_parameters = self.network.state_dict()
              best_f1_score = cur_f1_score
        # print(f"Best val => {best_valid_loss}")
        print(f"Best val f1=> {best_f1_score}")
        if best_parameters != None:
          self.network.load_state_dict(best_parameters)
        

        return "Model trained successfully"
    def setup(self):
      self.dl_train = self.create_dataloader(sentences = self.dataset_training.get('sentences'),
                                  tags = self.dataset_training.get('tags'), 
                                )
      self.dl_validate = self.create_dataloader(sentences = self.dataset_validation.get('sentences'), 
                                      tags = self.dataset_validation.get('tags'),
                                    )

    def create_dataloader(self, sentences, tags):
      
      data_reader = NERDataset(
          sentences = sentences, 
          tags = tags,
          transformer_tokenizer = self.transformer_tokenizer, 
          transformer_config = self.transformer_config,
          TAGS_DICT = self.TAGS
          )

      data_loader = torch.utils.data.DataLoader(
          data_reader, batch_size = self.train_batch_size, num_workers = 2, pin_memory=True
      )

      return data_loader
    def train(self,):
      self.network.train()    
      final_loss = 0.0
      
      for dl in tqdm(self.dl_train, total=len(self.dl_train)):

          self.optimizer.zero_grad()
          outputs = self.network(**dl)
          loss = self.compute_loss(outputs, 
                              dl.get('target_tags'),
                              dl.get('masks'), 
                              )
          # loss = self.network(**dl)
          loss.backward()
          # torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.clip)
          self.optimizer.step()
          self.scheduler.step()
          final_loss += loss.item()

      return final_loss / len(self.dl_train)
    def validate(self):
        self.network.eval()
        final_loss = 0.0

        for dl in tqdm(self.dl_validate, total=len(self.dl_validate)):
            
            outputs = self.network(**dl)
            loss = self.compute_loss(outputs, 
                                dl.get('target_tags'),
                                dl.get('masks'), 
                                )
            # loss = self.network(**dl)
            final_loss += loss.item()
        
        return final_loss / len(self.dl_validate)   
    def compute_loss(self, preds, target_tags, masks,):
        # ----------- CRF -----------------
        # preds = preds.permute(1, 0, 2).to(self.device)
        # target_tags = target_tags.permute(1, 0).type(torch.long).to(self.device)
        # target_tags = target_tags.type(torch.uint8)
        # masks = masks.permute(1, 0).type(torch.uint8).to(self.device)
        # print(target_tags, masks)
        # masks = masks.type(torch.uint8)
        # print(preds.shape, target_tags.shape, masks.shape)
        # loss = self.model_crf(preds, target_tags, mask=masks)
        # ----------- CRF -----------------
        active_loss = masks.view(-1) == 1

        active_logits = preds.view(-1, self.n_tags)
        active_labels = torch.where(
            active_loss,
            target_tags.view(-1),
            torch.tensor(self.criterion.ignore_index).type_as(target_tags)
        )
        active_labels = torch.as_tensor(active_labels, device = self.device, dtype = torch.long)
        loss = self.criterion(active_logits, active_labels)
        return loss
    def evaluate_performance(self, dataset):
        
        tags_predicted = self.predict(dataset.get('sentences'))
        tags = list(set(flatten(dataset.get('tags'))))
        tags.remove('O')
        
        f1 = compute_f1_scores(y_pred = tags_predicted, 
                               y_true = dataset.get('tags'),
                               labels = tags,
                               average = None
                               )
        df = list(zip(tags, f1[2]))
        df = pd.DataFrame(df, columns = ['Level', 'F1-Score'])    
        
        f1_micro = compute_f1_scores(y_pred = tags_predicted, 
                                     y_true = dataset.get('tags'),
                                     labels = tags,
                                     average = 'micro')
        f1_micro = pd.DataFrame({'Level' : ['AVG_MICRO'], 'F1-Score': [f1_micro[2]]})
        df = df.append(f1_micro)

        f1_macro = compute_f1_scores(y_pred = tags_predicted, 
                                     y_true = dataset.get('tags'),
                                     labels = tags,
                                     average = 'macro')
        f1_macro = pd.DataFrame({'Level' : ['AVG_MACRO'], 'F1-Score': [f1_macro[2]]})
        df = df.append(f1_macro)
      
        # -------------
        print(classification_report(dataset.get('tags'), tags_predicted))
        return df 
    def predict(self, sentences):
      self.network.eval()

      tag_fill = ["O"]
      tags_dummy = [tag_fill * len(sent) for sent in sentences]
      
      dl_test = self.create_dataloader(sentences=sentences, tags = tags_dummy)

      predictions = []
      with torch.no_grad():
        for i, dl in enumerate(dl_test): 
          outputs = self.network(**dl)   
          # outputs = self.network.predict(**dl)   
          # print(outputs)
          for i in range(len(outputs)):
              preds = [self.TAGS_NUM[item] for item in outputs[i].argmax(-1).cpu().numpy()]
              # preds = outputs[i]
              preds = [prediction for prediction, offset in zip(preds, dl.get('offsets')[i]) if offset]
              preds = preds[1:-1]
              predictions.append(preds)

      return predictions

In [5]:
nlp = spacy.load("en_core_web_sm")

SEED = 1234
 
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


full_dataset = pd.read_csv('./datasets/combined_24_02_2021_conll2003.csv', encoding='utf-8') 
train, valid, test = np.split(full_dataset.sample(frac=1, random_state=42), 
                       [int(.7*len(full_dataset)), int(.85*len(full_dataset))])

get_sent_data = lambda df, field: [str(item).split() for item in df[field]]
get_label_data = lambda df, field: [str(item).split() for item in df[field]]
train_data = {
    'sentences': get_sent_data(train, 'sent'), 
    'tags': get_label_data(train, 'named_entity')
}
valid_data = {
    'sentences': get_sent_data(valid, 'sent'), 
    'tags': get_label_data(valid, 'named_entity')
}
test_data = {
    'sentences': get_sent_data(test, 'sent'), 
    'tags': get_label_data(test, 'named_entity')
}

NER_TYPES = {
    "O": 0,
    'B-a1_be_have_do_in_the_past':1,
    'B-a1_can':2,
    'B-a1_comparative_exept':3,
    'B-a1_comparative_long':4,
    'B-a1_comparative_short':5,
    'B-a1_future_simple':6,
    'B-a1_have_has_got':7,
    'B-a1_past_simple_irreg':8,
    'B-a1_past_simple_reg':9,
    'B-a1_possesive_s_sing':10,
    'B-a1_possessive_s_plurar':11,
    'B-a1_present_continuous_act_rn':12,
    'B-a1_present_simple_3d_pers':13,
    'B-a1_present_simple_reg_act':14,
    'B-a1_special_questions':15,
    'B-a1_superlative_exept':16,
    'B-a1_superlative_long':17,
    'B-a1_superlative_short':18,
    'B-a1_there_is_am_are':19,
    'B-a1_there_was_were':20,
    'B-a1_there_will_be':21,
    'B-a1_to_be_future_will_be':22,
    'B-a1_to_be_past_was_were':23,
    'B-a1_to_be_present_is_am_are':24,
    'B-a1_want_would_like_to':25,
    # inside
    'I-a1_can':26,
    'I-a1_comparative_exept':27,
    'I-a1_comparative_long':28,
    'I-a1_comparative_short':29,
    'I-a1_future_simple':30,
    'I-a1_have_has_got':31,
    'I-a1_past_simple_irreg':32,
    'I-a1_past_simple_reg':33,
    'I-a1_possesive_s_sing':34,
    'I-a1_possessive_s_plurar':35,
    'I-a1_present_continuous_act_rn':36,
    'I-a1_present_simple_3d_pers':37,
    'I-a1_present_simple_reg_act':38,
    'I-a1_special_questions':39,
    'I-a1_superlative_exept':40,
    'I-a1_superlative_long':41,
    'I-a1_superlative_short':42,
    'I-a1_there_is_am_are':43,
    'I-a1_there_was_were':44,
    'I-a1_there_will_be':45,
    'I-a1_to_be_future_will_be':46,
    'I-a1_to_be_past_was_were':47,
    'I-a1_to_be_present_is_am_are':48,
    'I-a1_want_would_like_to':49,
    'I-a1_be_have_do_in_the_past':50,
}
 
tag_scheme = {
  # "O": 0,
  'B-a1_be_have_do_in_the_past':1,
  'B-a1_can':2,
  'B-a1_comparative_exept':3,
  'B-a1_comparative_long':4,
  'B-a1_comparative_short':5,
  'B-a1_future_simple':6,
  'B-a1_have_has_got':7,
  'B-a1_past_simple_irreg':8,
  'B-a1_past_simple_reg':9,
  'B-a1_possesive_s_sing':10,
  'B-a1_possessive_s_plurar':11,
  'B-a1_present_continuous_act_rn':12,
  'B-a1_present_simple_3d_pers':13,
  'B-a1_present_simple_reg_act':14,
  'B-a1_special_questions':15,
  'B-a1_superlative_exept':16,
  'B-a1_superlative_long':17,
  'B-a1_superlative_short':18,
  'B-a1_there_is_am_are':19,
  'B-a1_there_was_were':20,
  'B-a1_there_will_be':21,
  'B-a1_to_be_future_will_be':22,
  'B-a1_to_be_past_was_were':23,
  'B-a1_to_be_present_is_am_are':24,
  'B-a1_want_would_like_to':25,
  # inside
  'I-a1_can':26,
  'I-a1_comparative_exept':27,
  'I-a1_comparative_long':28,
  'I-a1_comparative_short':29,
  'I-a1_future_simple':30,
  'I-a1_have_has_got':31,
  'I-a1_past_simple_irreg':32,
  'I-a1_past_simple_reg':33,
  'I-a1_possesive_s_sing':34,
  'I-a1_possessive_s_plurar':35,
  'I-a1_present_continuous_act_rn':36,
  'I-a1_present_simple_3d_pers':37,
  'I-a1_present_simple_reg_act':38,
  'I-a1_special_questions':39,
  'I-a1_superlative_exept':40,
  'I-a1_superlative_long':41,
  'I-a1_superlative_short':42,
  'I-a1_there_is_am_are':43,
  'I-a1_there_was_were':44,
  'I-a1_there_will_be':45,
  'I-a1_to_be_future_will_be':46,
  'I-a1_to_be_past_was_were':47,
  'I-a1_to_be_present_is_am_are':48,
  'I-a1_want_would_like_to':49,
  'I-a1_be_have_do_in_the_past':50,
}

tag_scheme = list(tag_scheme.keys())


In [ ]:
len(train_data['sentences']), len(valid_data['sentences'])

(490, 105)

In [6]:
model = NERExperiment(
  dataset_training = train_data,
  dataset_validation = valid_data,
  TAGS = NER_TYPES, 
  model_name = 'roberta-base',
  dropout=0.1,
  train_batch_size=4,
  # grad_clip=100,
  optimizer_class=AdamW
)
# RAdam, Lookahead, QHAdamW, AdamP


In [7]:
model.network.load_state_dict(torch.load("./my_nerda/roberta-base-0.777-0.268-2-22-21.pt"))

<All keys matched successfully>

In [ ]:
model.network = model.network.to('cuda')

In [ ]:
model.experiment(epochs=7, 
                 learning_rate = 4e-6
                 )

In [ ]:
model.evaluate_performance(test_data)

['I-a1_possessive_s_plurar', 'B-a1_to_be_future_will_be', 'I-a1_past_simple_reg', 'B-a1_past_simple_irreg', 'I-a1_there_was_were', 'B-a1_present_continuous_act_rn', 'B-a1_have_has_got', 'I-a1_present_simple_reg_act', 'I-a1_there_is_am_are', 'I-a1_be_have_do_in_the_past', 'I-a1_future_simple', 'B-a1_there_was_were', 'B-a1_there_is_am_are', 'I-a1_past_simple_irreg', 'B-a1_comparative_exept', 'B-a1_to_be_present_is_am_are', 'I-a1_present_simple_3d_pers', 'B-a1_want_would_like_to', 'I-a1_to_be_past_was_were', 'I-a1_comparative_long', 'B-a1_superlative_short', 'B-a1_possessive_s_plurar', 'I-a1_have_has_got', 'I-a1_want_would_like_to', 'B-a1_be_have_do_in_the_past', 'B-a1_present_simple_3d_pers', 'I-a1_to_be_future_will_be', 'I-a1_superlative_exept', 'I-a1_present_continuous_act_rn', 'B-a1_can', 'I-a1_to_be_present_is_am_are', 'B-a1_to_be_past_was_were', 'B-a1_possesive_s_sing', 'I-a1_superlative_short', 'I-a1_can', 'B-a1_comparative_long', 'B-a1_present_simple_reg_act', 'B-a1_comparative_sh

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Level,F1-Score
0,I-a1_possessive_s_plurar,0.666667
1,B-a1_to_be_future_will_be,1.000000
2,I-a1_past_simple_reg,0.648649
3,B-a1_past_simple_irreg,0.714286
4,I-a1_there_was_were,1.000000
5,B-a1_present_continuous_act_rn,0.875000
6,B-a1_have_has_got,0.833333
7,I-a1_present_simple_reg_act,0.813187
8,I-a1_there_is_am_are,1.000000
9,I-a1_be_have_do_in_the_past,0.000000


In [ ]:
# torch.save(model.network.state_dict(), "./my_nerda/roberta-base-0.752-0.257-6-14-21.pt")
# model.network.load_state_dict(torch.load("./my_nerda/roberta-base-0.777-0.268-2-22-21.pt"))

In [ ]:
# 0.752-0.257 - seed 42
# 0.757-0.32

In [ ]:
# torch.save(model.network.state_dict(), "./my_nerda/roberta-base-0.77-loss-0.278.pt")

In [8]:
# torch.multiprocessing.set_start_method('spawn', force=True)
sentences_orig = [
        #  'Are there many monkeys in the jungles?',
        #  'DeepSpeed is a deep learning library on top of PyTorch that makes training models at extreme-scale efficient and easy for everyone.', 
        #  'DeepSpeed offers powerful training features for data scientists training on massive supercomputers as well as those training on low-end clusters or even on a single GPU.',
        #  'There are other ways that I am using self - care to tend to my mental health.',
        #  "What's more, essential workers are risking their lives so that we can have our necessities.",
        #  'While all of this is happening, the luxury travel market is growing and becoming even more exclusive than it already was.',
        #  "Let's say you live in iowa and you have a bang coming on once a month.",
        #   "The thing is, I might even like what he's saying and maybe even agree with him.",
        #  "Sometimes his mouth moves like he's talking to someone, but there's no one else there.",
        #  "He's just sitting there outside her house in normal civilian clothes, all alone.",
        #  "He doesn't know where he's going; he just goes.",
        #  "He's still smiling, looking at Sean with kind, knowing eyes.",
        #  "He follows Joey into his apartment, past a living area where two other guys are watching TV.",
        #  "Sean accidentally shoulders the elevator wall as he's walking out, but doesn't acknowledge it.",
        #  "He feels comfortable giving away his address, whereas Sean would never dare give him his.",
        #  "It's the same address from the police files, just two blocks away.",
        #  "He's smiling in the picture, even though you're not supposed to do that for ID photos.",
        #  'When her friends asked her about it, she told them that she was having trouble with some "horse syndicate" people.',
        #  "In a letter addressed to her husband, Renee wrote that she was unhappy in her marriage and was contemplating getting a divorce.",
        #  "Terry was coming toward me, a huge grin plastered on his face - as he pumped his legs on his unicycle.",
        #  "We met at Piedmont Park, where Terry had suggested we could ride bikes.",
        #  "What I had feared ended up coming true - my hair smelled like Terry's discount ham for weeks.",
        #  "Perseverance has a large amount of data in its memory banks which it is gradually offloading to Earth.",
        #  "Nasa is promising more in the next few days.",
        #  "It shows the robot heading down to the ground on Thursday to make its landing.",
        #  "It was acquired by the rocket cradle that placed the vehicle on the surface.",
        #  "Perseverance has been put in a near-equatorial Martian crater known as Jezero where it will search for signs of past microbial life.",
        #  "You can see the dust kicked up by the engines.",
        #  "We're probably about 2m or so above the surface of Mars.",
        #  "And then the curly electrical umbilical that is taking all of the electrical signals from the descent stage down to the computer inside the belly of the rover, the ones and zeros that represent this image.",
        #  "Engineers report Perseverance to be in good health, as they gradually commission its systems.",
        #  "Even now, with just this limited first release of pictures, there were fascinating rocks to discuss, she told reporters.",
        #  "The $2.7bn (£1.9bn) robot is the fifth rover to be put on Mars by Nasa.",
        #  "As well as searching for signs of life, Perseverance's other key objective is to select and package rock samples that can be brought back to Earth laboratories by later missions.",
        # "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?",
        # "When did the Scholastic Magazine of Notre dame begin publishing?",
        "OK, it's not marketing; there's a very interesting car concept here, but the marketing part is not what I want to talk about here.",
        "What I want to talk about is, as background, is the idea that cars are art.",
        "This is actually quite meaningful to me, because car designers tend to be a little bit low on the totem pole -- we don't do coffee table books with just one lamp inside of it -- and cars are thought so much as a product that it's a little bit difficult to get into the aesthetic side under the same sort of terminology that one would discuss art.",
        "Now at this point you're going to see a picture of Michelangelo.",
        "And as long as we can solve the problems of automobiles, and I believe we can, with fuel cells or hydrogen, like BMW is really hip on, and lots of other things, then I think we can look past that and try and understand why this hook is in many of us -- of this car-y-ness -- and what that means, what we can learn from it.",
        "Cars are an expansion of yourself: they take your thoughts, your ideas, your emotions, and they multiply it -- your anger, whatever.",
        "And they hadn't done anything because -- kind of, like in car business, engineers are there to solve problems, and we were asking them to create a problem."
         ]
sentences = [[str(i)for i in nlp(str(sent))] for sent in sentences_orig]
predicts = model.predict(sentences)
# sentences

In [9]:
def get_entity_from_sent(sent, label):
    prev_label = label[0][2:]
    # sent = normal_nlp(str(sent))
    sent = [str(token) for token in sent]
    relu = lambda x: x if x > 0 else 0
    accum = []
    ent_len = 0
    start = 0
    for i, tok in enumerate(label):
      tok = tok[2:]
      ent_len += 1
      if tok != label[relu(i-1)][2:]:
        prev_ent = label[relu(i-1)][2:]
        if ent_len > 1 and prev_ent != '':
          accum.append({
              'ent': prev_ent,
              'start': relu(start),
              'end': relu(i)
              })
        ent_len = 0
        start = i
 
    full_sent =  str(" ".join(sent))
    result = {
          'text': full_sent,
          'ents': [],
          'title': None
      }
    for ent in accum:
      sub_ent = " ".join(sent[ent['start']:ent['end']])
      start = full_sent.index(sub_ent)
      end = start + len(sub_ent)
      result['ents'].append(
          {'start': start, 'end': end, 'label': ent['ent']}
      )
    return result

def get_entity_from_sent_lite(sent, label):
    prev_label = label[0][2:]
    sent = [str(token) for token in sent]
    relu = lambda x: x if x > 0 else 0
    accum = []
    ent_len = 0
    start = 0
    for i, tok in enumerate(label):
      tok = tok[2:]
      ent_len += 1
      if tok != label[relu(i-1)][2:]:
        prev_ent = label[relu(i-1)][2:]
        if ent_len > 1 and prev_ent != '':
          accum.append(prev_ent)
        ent_len = 0
        start = i
        
    return accum

In [10]:
for sent, pred in zip(sentences, predicts):
  entities = get_entity_from_sent(sent, pred)
  # print(entities) 
  # break
  html = displacy.render(entities, style="ent", manual=True, jupyter=True)

# Create dataset

In [ ]:
import spacy

In [ ]:
!ls medium_articles/

all-medium-2-22-21	med_5_2_21	 med_gen_4_23_21.csv
all-medium-2-22-21.csv	med_5_2_21.zip	 med_gen_4_23_21.zip
all-medium-2-22-21.zip	med_gen_4_23_21


In [ ]:
import json
import os
# from natsort import natsorted, ns
import pandas as pd
import numpy as np
# from progress.bar import ChargingBar


class CreateCSV:
    def __init__(self, source, dest, text_only=False):
        self.source = source
        self.dest = dest
        self.columns = ["title", "text", 'topic']
        self.df = pd.DataFrame(columns=self.columns)
        self.text_only = text_only
        self.read_medium()

    def article_content(self, json_data):
        only_text = ""
        for item in json_data['data']:
            if item['type'] == "sentence":
                content = item['content']
                only_text += content
        return only_text

    def get_first_img(self, json_data):
        img_path = ""
        for item in json_data["data"]:
            if item['type'] == 'image':
                img_path = item['content']
                break
        return img_path

    def read_medium(self):
        all_articles = os.listdir(self.source)
        print(len(all_articles))
        for article in all_articles:
            row = self.json_to_row(f"{self.source}{article}")
            if row != 0:
                self.df = self.df.append(row, ignore_index=True)
                # bar.next()
            else:
                print(article)

        self.df[self.columns].to_csv(self.dest, index=False)

    def json_to_row(self, json_path):
        json_data = json.load(open(f'{json_path}', "r", encoding="utf-8"))

        text = self.article_content(json_data)
        img = self.get_first_img(json_data)
        if img == "" and not self.text_only:
            return 0
        row = {}
        row["title"] = json_data["title"]
        row["text"] = text
        row["topic"] = json_data['topic']
        row['uuid'] = json_data["uuid"]
        row['link'] = json_data["link"]
        row['img'] = img
        return row

In [ ]:
med_to_dataframe = CreateCSV("./medium_articles/med_5_2_21/", "./medium_articles/med_5_2_21.csv", text_only=True)

639


In [ ]:
all_medium = pd.read_csv("./medium_articles/med_5_2_21.csv", encoding='utf-8')
all_medium

,title,text,topic
0,GDP Growth Rate Shows Biden's Spending Plans W...,Inflation for the first quarter was nearly 4%....,economy
1,Incentive Models Deceive Us,"A few days ago, I found myself browsing websit...",economy
2,The 10 Most Important Lines From President Bid...,The most amusing shot in Wednesday night's bro...,economy
3,Stupid In Love,At home waiting on a call that never comes A l...,fiction
4,How I accidentally meditated for 90 days straight,I hate meditation - or at least I thought I di...,mindfulness
...,...,...,...
634,Will Mindfulness Mend Your Broken Heart?,"Mindfulness can't cure heartbreak. That, you d...",mindfulness
635,'A Perfect Positive Storm': Bonkers Dollars fo...,By Shira Ovide,economy
636,How to Co-Create With the Universe,"In March 2019, I attended a writing workshop i...",mindfulness
637,4 Keys to Successful Social Media Networking i...,Whether you're a novice or a seasoned professi...,marketing


In [ ]:
!python -m spacy download en_core_web_md

In [12]:
import re
import en_core_web_sm
# nlp = en_core_web_sm.load()
nlp = spacy.load("en_core_web_sm")

def pretty_text(dirty_text):
    bad_chars = {
        "’":"'",
        "‘": "'",
        "“": '"',
        "”": '"',
        "—": "-",
        # "…": "...",
        "–": "-",
        '--': '-',
        "&gt;": ' ',
        "\n": " "
    }
    new_text = str(dirty_text).strip()
    for bad_char in bad_chars:
        new_text = new_text.replace(bad_char, bad_chars[bad_char])
                
    _replace_whitespace_ = re.compile(r"\s+")
    new_text = _replace_whitespace_.sub(" ", new_text).strip()

    _replace_any_link_ = re.compile(r'(http\S+)')
    new_text = _replace_any_link_.sub(" ", new_text).strip()

    # _insert_whitespace_letter_ = re.compile(r'([\.\?\!,])(\w{1})')
    # new_text = _replace_multiple_whitespaces.sub( r'\1 \2', new_text).strip()
    new_text = re.sub(r'([\.\?\!,])(\w{1})', r'\1 \2', new_text)

    _replace_multiple_whitespaces = re.compile(r'\s{1,}')
    new_text = _replace_multiple_whitespaces.sub(" ", new_text).strip()

    return new_text

In [ ]:
# medium_sentences_dataset = pd.DataFrame(columns=['sentence', 'topic', 'tenses'])

In [ ]:
def create_medium_sentences_dataset(texts_dataframe, destination="temp.csv"):
  sentences = []
  topics = []
  tenses = []

  for i in range(len(texts_dataframe)):
    topic = texts_dataframe['topic'][i]
    text = nlp(pretty_text(texts_dataframe['text'][i])).sents
    # row = {}
    # row['sentence'] = ''
    # row['topic'] = topic
    # row['tenses'] = ""
    for sent in text:
      # row['sentence'] = sent
      # medium_sentences_dataset = medium_sentences_dataset.append(row, ignore_index=True)
      sentences.append(sent)
      topics.append(topic)
      tenses.append("")
  data = {
      'sentence': sentences, 
      'topic': topics, 
      'tenses': tenses
  }
  medium_sentences_dataset = pd.DataFrame(data=data)
  medium_sentences_dataset.to_csv(destination, index=False)

In [ ]:
def create_new_york_sentences_dataset(texts_dataframe, destination=None):
  assert destination != False

  for i in range(len(texts_dataframe)):
    topic = texts_dataframe['list_name_encoded'][i]
    text = nlp(pretty_text(texts_dataframe['text'][i])).sents
    row = {}
    row['sentence'] = ''
    row['topic'] = topic
    row['tenses'] = ""
    for sent in text:
      row['sentence'] = sent
      sentences_dataset = sentences_dataset.append(row, ignore_index=True)

  sentences_dataset.to_csv(destination, index=False)

In [ ]:
create_medium_sentences_dataset(all_medium, destination="./med_5_2_21_medium_sentences_dataset.csv")

In [ ]:
medium_sentences_dataset = pd.read_csv("./med_5_2_21_medium_sentences_dataset.csv",)
len(medium_sentences_dataset)

39182

In [ ]:
medium_sentences_dataset = medium_sentences_dataset[(medium_sentences_dataset['sentence'].duplicated() == False) & (medium_sentences_dataset['sentence'].isna() == False)]
medium_sentences_dataset = medium_sentences_dataset[medium_sentences_dataset['sentence'].str.split().apply(len) > 5]

In [ ]:
# medium_sentences_dataset.to_csv("./med_5_2_21_medium_sentences_dataset.csv",index=False)
medium_sentences_dataset = pd.read_csv("./med_5_2_21_medium_sentences_dataset.csv")

# Predict tenses

In [ ]:
nlp = spacy.load("en_core_web_sm")

def split_into_tokens(sent):
  return [str(i) for i in nlp(str(sent))]

def predict_dataset(medium_sentences_dataset=None):
  assert not medium_sentences_dataset is None

  data_chunk = 1000
  data_parts = len(medium_sentences_dataset) // data_chunk
  full_parts = data_chunk * data_parts
  for step in range(data_parts):
    sents = [str(item) for item in medium_sentences_dataset[step*data_chunk:(step+1)*data_chunk]['sentence'].to_list()]
    # sents = [[str(i)for i in nlp(str(sent))] for sent in tqdm(sents)]
    with ProcessPoolExecutor(max_workers=4) as executor:
      sents = list(executor.map(split_into_tokens, sents))

    predicted_tenses = model.predict(sents)
    entities = [get_entity_from_sent_lite(sent, pred) for sent, pred in zip(sents, predicted_tenses)]
    
    for i in range(len(entities)):
      dataframe_index = i + step * data_chunk
      ents = entities[i]
      if len(ents) > 0:
        medium_sentences_dataset['tenses'][dataframe_index] = ' '.join(ents)
      else:
        medium_sentences_dataset['tenses'][dataframe_index] = 'not_found'

  sents = [str(item) for item in medium_sentences_dataset[full_parts:]['sentence'].to_list()]
  # sents = [[str(i)for i in nlp(str(sent))] for sent in tqdm(sents)]
  with ProcessPoolExecutor(max_workers=4) as executor:
    sents = list(executor.map(split_into_tokens, sents))

  predicted_tenses = model.predict(sents)
  entities = [get_entity_from_sent_lite(sent, pred) for sent, pred in zip(sents, predicted_tenses)]

  for i in range(len(entities)):
    dataframe_index = i + full_parts
    ents = entities[i]
    if len(ents) > 0:
      medium_sentences_dataset['tenses'][dataframe_index] = ' '.join(ents)
    else:
      medium_sentences_dataset['tenses'][dataframe_index] = 'not_found'
  
  medium_sentences_dataset.to_csv("./comb_med_2119_6_17_21_sentences_labeled.csv.csv", index=False)
    
    

In [ ]:
# if __name__ == "__main__":
#   predict_dataset()

# predict questions grammar

In [ ]:
nlp = spacy.load("en_core_web_sm")
question_dataset = pd.read_csv("/content/drive/My Drive/collab_sandbox/question_generation/question_generation_patil_suraj/datasets/squad_qa_qg_6_7_21.csv")
question_dataset = question_dataset[question_dataset['task'] == 'generate_question'].reset_index()
question_dataset['tenses'] = ''
question_dataset = question_dataset.drop(columns=['index'])
question_dataset

,source_text,target_text,task,context,answer,question,tenses
0,"generate question: Architecturally, the school...",To whom did the Virgin Mary allegedly appear i...,generate_question,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,To whom did the Virgin Mary allegedly appear i...,
1,"generate question: Architecturally, the school...",What is in front of the Notre Dame Main Buildi...,generate_question,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,What is in front of the Notre Dame Main Building?,
2,"generate question: Architecturally, the school...",The Basilica of the Sacred heart at Notre Dame...,generate_question,"Architecturally, the school has a Catholic cha...",the Main Building,The Basilica of the Sacred heart at Notre Dame...,
3,"generate question: Architecturally, the school...",What is the Grotto at Notre Dame? </s>,generate_question,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,What is the Grotto at Notre Dame?,
4,"generate question: Architecturally, the school...",What sits on top of the Main Building at Notre...,generate_question,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,What sits on top of the Main Building at Notre...,
...,...,...,...,...,...,...,...
87378,generate question: Kathmandu is located in the...,What river is south of Kathmandu? </s>,generate_question,Kathmandu is located in the northwestern part ...,Bagmati,What river is south of Kathmandu?,
87379,generate question: Kathmandu is located in the...,What geographic portion of the Kathmandu Valle...,generate_question,Kathmandu is located in the northwestern part ...,northwestern,What geographic portion of the Kathmandu Valle...,
87380,generate question: Kathmandu is located in the...,About how many feet above sea level is Kathman...,generate_question,Kathmandu is located in the northwestern part ...,"4,600",About how many feet above sea level is Kathmandu?,
87381,generate question: Kathmandu is located in the...,How large is Kathmandu in terms of square mile...,generate_question,Kathmandu is located in the northwestern part ...,19.56,How large is Kathmandu in terms of square miles?,


In [ ]:
question_dataset['answer'][3]

'a copper statue of Christ'

In [19]:
nlp = spacy.load("en_core_web_sm", disable=['ner', 'tagger', 'parser', 'textcat'])
import time

def split_into_tokens(sent):
  return [str(i) for i in nlp(str(sent))]

def predict_dataset_questions(question_dataset=None, field=None):
  assert not question_dataset is None
  assert field
  """ estimated time 1 hour 1 min """
  data_chunk = 1000
  data_parts = len(question_dataset) // data_chunk
  full_parts = data_chunk * data_parts
  total_time = 0
  prev_now = time.time()
  for step in range(data_parts):
    sents = [str(item) for item in question_dataset[step*data_chunk:(step+1)*data_chunk][field].to_list()]
    with ProcessPoolExecutor(max_workers=4) as executor:
      sents = list(executor.map(split_into_tokens, sents))

    predicted_tenses = model.predict(sents)
    entities = [get_entity_from_sent_lite(sent, pred) for sent, pred in zip(sents, predicted_tenses)]
    
    time_diff = time.time() - prev_now
    total_time += time_diff
    prev_now = time.time()
    average_time = total_time / (step+1)
    remain_time = (data_parts - (step+1)) * average_time // 60
    print(f"Remain time min: {remain_time}")
    
    for i in range(len(entities)):
      dataframe_index = i + step * data_chunk
      ents = entities[i]
      if len(ents) > 0:
        question_dataset['tenses'][dataframe_index] = ' '.join(ents)
      else:
        question_dataset['tenses'][dataframe_index] = 'not_found'

  sents = [str(item) for item in question_dataset[full_parts:][field].to_list()]
  with ProcessPoolExecutor(max_workers=4) as executor:
    sents = list(executor.map(split_into_tokens, sents))

  predicted_tenses = model.predict(sents)
  entities = [get_entity_from_sent_lite(sent, pred) for sent, pred in zip(sents, predicted_tenses)]

  for i in range(len(entities)):
    dataframe_index = i + full_parts
    ents = entities[i]
    if len(ents) > 0:
      question_dataset['tenses'][dataframe_index] = ' '.join(ents)
    else:
      question_dataset['tenses'][dataframe_index] = 'not_found'
  
  question_dataset.to_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled.csv", index=False)
    

In [ ]:
# comb_med_2119 = pd.read_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences.csv")
# if __name__ == "__main__":
#   predict_dataset_questions(question_dataset=comb_med_2119, field='sentence')

In [23]:
comb_med = pd.read_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled.csv")
comb_med
# comb_med[comb_med['tenses'] != 'not_found'].reset_index(drop=True).to_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled.csv",index=False)

,sentence,topic,tenses
0,"It pollutes the environment, it's limited in s...",science,a1_present_simple_3d_pers a1_to_be_present_is_...
1,"Of course, nowadays electric cars are undoubte...",science,a1_to_be_present_is_am_are a1_comparative_long...
2,"Yet, it's worth talking about biofuels, as the...",science,a1_to_be_present_is_am_are a1_present_simple_r...
3,"While there are many types of biofuels, we'll ...",science,a1_there_is_am_are a1_future_simple a1_superla...
4,"There are also biofuels for planes and boats, ...",science,a1_there_is_am_are
...,...,...,...
107388,"In fact, I'd argue it's our duty to encourage ...",travel,a1_to_be_present_is_am_are
107389,It also leverages global talent in service of us,travel,a1_present_simple_3d_pers
107390,all - look no further than the San Francisco B...,travel,a1_to_be_present_is_am_are
107391,"It's not easy, but not impossible; in fact, no...",travel,a1_to_be_present_is_am_are a1_present_simple_r...


In [25]:
!readlink -f ./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled.csv

/content/drive/My Drive/collab_sandbox/NER/ner_tenses_recognition/datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled.csv


In [ ]:
import pandas as pd

question_dataset = pd.read_csv("./question_dataset_tenses_6_7_21_fix.csv")[['tenses', 'question']] 

for i in range(50):
  question = question_dataset['question'][i]
  tenses = question_dataset['tenses'][i]
  print(f"{tenses} -> {question}")

a1_past_simple_reg -> To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
a1_to_be_present_is_am_are -> What is in front of the Notre Dame Main Building?
not_found -> The Basilica of the Sacred heart at Notre Dame is beside to which structure?
a1_to_be_present_is_am_are -> What is the Grotto at Notre Dame?
not_found -> What sits on top of the Main Building at Notre Dame?
not_found -> When did the Scholastic Magazine of Notre dame begin publishing?
a1_possesive_s_sing -> How often is Notre Dame's the Juggler published?
a1_to_be_present_is_am_are -> What is the daily student paper at Notre Dame called?
not_found -> How many student news papers are found at Notre Dame?
not_found -> In what year did the student paper Common Sense begin publication at Notre Dame?
a1_to_be_present_is_am_are -> Where is the headquarters of the Congregation of the Holy Cross?
a1_to_be_present_is_am_are -> What is the primary seminary of the Congregation of the Holy Cross?
a1_to_be_present_i

In [ ]:
medium_sentences_dataset_labeled = pd.read_csv("./medium_sentenses_labeled_all__4_23_21_80479__Sun May  2 01:18:31 2021_1619918311.231216.csv", encoding='utf-8')
med_5_2_21_medium_sentences_dataset_labeled = pd.read_csv("./med_5_2_21_medium_sentences_dataset_labeled__Sun May  2 19:36:07 2021_1619984167.8032556.csv", encoding='utf-8')
medium_sentenses_labeled_all = pd.concat([medium_sentences_dataset_labeled, med_5_2_21_medium_sentences_dataset_labeled], ignore_index=True)

In [ ]:
len(medium_sentenses_labeled_all[medium_sentenses_labeled_all['sentence'].duplicated()])

1357

In [ ]:
# med_5_2_21_medium_sentences_dataset_labeled = pd.read_csv("./med_5_2_21_medium_sentences_dataset_labeled.csv", encoding='utf-8')
# med_5_2_21_medium_sentences_dataset_labeled = med_5_2_21_medium_sentences_dataset_labeled[(med_5_2_21_medium_sentences_dataset_labeled['tenses'] != 'not_found') & (med_5_2_21_medium_sentences_dataset_labeled['sentence'].duplicated() == False)]
# med_5_2_21_medium_sentences_dataset_labeled.to_csv("./med_5_2_21_medium_sentences_dataset_labeled.csv", index=False)

In [ ]:
# medium_sentenses_labeled_all = medium_sentenses_labeled_all[(medium_sentenses_labeled_all['tenses'] != 'not_found') & (medium_sentenses_labeled_all['sentence'].duplicated() == False)]
# medium_sentenses_labeled_all
# medium_sentenses_labeled_all.to_csv("medium_sentenses_labeled_all__5_2_21_88887.csv", index=False)

# Filter Meaningfull sentences

In [ ]:
def load_meaningful_sentences():
  meaningful_sentences_1 = pd.read_csv("./meaningful_classif/meaningful_sentences_classification__1001__4_25_21__5_34.csv")
  meaningful_sentences_1 = meaningful_sentences_1[(meaningful_sentences_1['Tag'] == 'yes') | (meaningful_sentences_1['Tag'] == 'no')]

  meaningful_sentences_2 = pd.read_csv("./meaningful_classif/meaningful_sentences_classification_2_801__4_27_21.csv")
  meaningful_sentences_2 = meaningful_sentences_2[(meaningful_sentences_2['Tag'] == 'yes') | (meaningful_sentences_2['Tag'] == 'no')]

  meaningful_sentences = pd.concat([meaningful_sentences_1, meaningful_sentences_2], ignore_index=True)

  return meaningful_sentences
  
meaningful_sentences = load_meaningful_sentences()

In [ ]:
meaningful_sentences['Tag'].value_counts()

yes    1054
no      744
Name: Tag, dtype: int64

In [ ]:
!pip freeze | grep spacy 

spacy==3.0.6
spacy-alignments==0.8.3
spacy-legacy==3.0.5
spacy-transformers==1.0.2


In [ ]:
!pip freeze | grep cupy

cupy-cuda101==7.4.0
cupy-cuda111==8.4.0


In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy[cuda101,transformers,lookups]
# !python -m spacy download en_core_web_trf
# !pip install spacy==3.0.3 \
# spacy-alignments==0.7.2 \
# spacy-legacy==3.0.1 \
# spacy-lookups-data==1.0.0 \
# spacy-transformers==1.0.1 \
# cupy-cuda111==8.4.0

In [ ]:
# !pip install spacy==3.0
# !python -m spacy download en_core_web_md
# !pip install spacy[transformers,cuda112]
# !pip install -U pip setuptools wheel
!pip install -U spacy[transformers]
!python -m spacy download en_core_web_trf importlib-metadata==3.3.0

In [ ]:
import spacy
from spacy.tokens import Doc, DocBin
spacy.require_gpu(0)

True

In [ ]:
!python -m spacy init fill-config ./spacy_textcat/gpu_accuracy_transformer.cfg ./spacy_textcat/base_gpu_accuracy_transformer.cfg

2021-04-27 18:55:13.448964: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
spacy_textcat/base_gpu_accuracy_transformer.cfg
You can now add your data and train your pipeline:
python -m spacy train base_gpu_accuracy_transformer.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("en_core_web_trf")


def make_docs(dataset):
  dataset = dataset.reset_index()
  dataset = [(str(sent), str(label)) for sent, label in zip(dataset['Text'].tolist(), dataset['Tag'].tolist())]
  # print(dataset)
  docs = []
  labels = {'yes': True, 'no': False}
  for doc, label in nlp.pipe(dataset, as_tuples=True):
    doc.cats["positive"] = int(labels[label])
    doc.cats["negative"] = int(not labels[label])
    docs.append(doc)
  return docs

num_texts = 1000
train_dataset, valid_dataset = train_test_split(meaningful_sentences, test_size=0.2, random_state=42)

train_docs = make_docs(train_dataset)
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./spacy_textcat/data/train.spacy")

valid_docs = make_docs(valid_dataset)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./spacy_textcat/data/valid.spacy")

In [ ]:
train_docs[0].cats

{'negative': 1, 'positive': 0}

In [ ]:
!python -m spacy train ./spacy_textcat/base_efficient_textcat_spacy.cfg --output ./spacy_textcat/base_efficient_textcat_spacy/ \
--paths.train ./spacy_textcat/data/train.spacy \
--paths.dev ./spacy_textcat/data/valid.spacy --gpu-id 0

## predict meaningful

In [ ]:
cat_nlp = spacy.load("./spacy_textcat/base_efficient_textcat_spacy/model-best/")
def predict_sent_meaningful(text):
  doc = cat_nlp(text)
  confidence = doc.cats['positive']
  if confidence >.5:
      print(f"positive {confidence} {text}")
  else:
      print(f"negative {confidence} {text}")

In [ ]:
# predict_sent_meaningful(", you only lose one point instead of being marked off for the entire problem.)This growth mindset relies on the notion that the most important thing a teacher can reinforce is to 'Fail first, then learn' (Eva).")
sentences = [
  "By failing, a student can learn a lot.",
  "For example, this type of problem-solving can be seen most commonly in math.",
  "I like cats"
]
for sent in sentences:
  predict_sent_meaningful(sent)


## Create medium dataset for controllable text generation 6 15 21

In [ ]:
pd.read_csv("./medium_articles/all-medium-2-22-21.csv")

In [ ]:
pd.read_csv("./medium_articles/med_gen_4_23_21.csv")

In [ ]:
pd.read_csv("./medium_articles/med_5_2_21.csv")

In [ ]:
def get_all_medium_articles():
  fields = ['title', 'text', 'topic']
  all_medium_2_22_21 = pd.read_csv("./medium_articles/all-medium-2-22-21.csv")[fields]
  med_gen_4_23_21 = pd.read_csv("./medium_articles/med_gen_4_23_21.csv")[fields]
  med_5_2_21 = pd.read_csv("./medium_articles/med_5_2_21.csv")[fields]

  merged_dataset  = pd.concat([all_medium_2_22_21, med_gen_4_23_21, med_5_2_21], ignore_index=True)
  merged_dataset = merged_dataset[merged_dataset['text'].duplicated() == False]
  merged_dataset = merged_dataset[merged_dataset['title'].duplicated() == False]
  return merged_dataset

In [ ]:
# merged_dataset = get_all_medium_articles()
# print(len(merged_dataset)) # 2119

2119


In [ ]:
merged_dataset.to_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_15_21_articles.csv", index=False)

In [13]:
comb_med_2119_6_15_21_articles = pd.read_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_15_21_articles.csv")

In [14]:
import time

def split_dataset_into_sentences(destination=None, dataset=None):
  """5 min 41 sec for 2119 texts"""
  assert destination
  assert not dataset is None

  sentences = []
  topics = []
  tenses = []
  sents_buffer = {}

  total_time = 0
  prev_now = time.time()
  data_len = len(dataset)
  for i in range(data_len):
    topic = dataset['topic'][i]
    text = nlp(pretty_text(dataset['text'][i])).sents
    for sent in text:
      sent = str(sent)
      if len(sent.split()) > 6 and sents_buffer.get(sent, True):
        sents_buffer[sent] = False
        sentences.append(sent)
        topics.append(topic)
        tenses.append("")
        
    if i % 10 == 0:
      time_diff = time.time() - prev_now
      total_time += time_diff
      prev_now = time.time()
      average_time = total_time / (i+1)
      remain_time = (data_len - (i+1)) * average_time // 60
      print(f"Remain time min: {remain_time} - {(i+1)} ~ { (i+1) / data_len}")

  data = {
      'sentence': sentences, 
      'topic': topics, 
      'tenses': tenses
  }
  medium_sentences_dataset = pd.DataFrame(data=data)
  medium_sentences_dataset.to_csv(destination, index=False)

In [ ]:
for i in range(15):
  print(pretty_text(comb_med_2119_6_15_21_articles['text'][i]))

In [15]:
split_dataset_into_sentences("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences.csv", comb_med_2119_6_15_21_articles)

Remain time min: 9.0 - 1 ~ 0.00047192071731949034
Remain time min: 8.0 - 11 ~ 0.005191127890514393
Remain time min: 8.0 - 21 ~ 0.009910335063709298
Remain time min: 7.0 - 31 ~ 0.0146295422369042
Remain time min: 7.0 - 41 ~ 0.019348749410099102
Remain time min: 8.0 - 51 ~ 0.02406795658329401
Remain time min: 8.0 - 61 ~ 0.02878716375648891
Remain time min: 8.0 - 71 ~ 0.03350637092968381
Remain time min: 8.0 - 81 ~ 0.038225578102878716
Remain time min: 8.0 - 91 ~ 0.04294478527607362
Remain time min: 8.0 - 101 ~ 0.04766399244926852
Remain time min: 7.0 - 111 ~ 0.052383199622463426
Remain time min: 7.0 - 121 ~ 0.05710240679565833
Remain time min: 7.0 - 131 ~ 0.06182161396885323
Remain time min: 7.0 - 141 ~ 0.06654082114204814
Remain time min: 7.0 - 151 ~ 0.07126002831524304
Remain time min: 7.0 - 161 ~ 0.07597923548843795
Remain time min: 7.0 - 171 ~ 0.08069844266163284
Remain time min: 7.0 - 181 ~ 0.08541764983482775
Remain time min: 6.0 - 191 ~ 0.09013685700802265
Remain time min: 6.0 - 2

In [16]:
comb_med_2119_6_15_21_sentences = pd.read_csv("./datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences.csv")

In [17]:
def filter_sentence_dataset(dataset=None):
  assert not dataset is None
  dataset['tokens'] = dataset['sentence'].apply(lambda s: len(str(s).split()))
  dataset = dataset[dataset['tokens'] > 6].drop(columns=['tokens']).reset_index(drop=True)

  return dataset

In [18]:
comb_med_2119_6_15_21_sentences = filter_sentence_dataset(comb_med_2119_6_15_21_sentences)
comb_med_2119_6_15_21_sentences

,sentence,topic,tenses
0,"It pollutes the environment, it's limited in s...",science,NaN
1,"Unsurprisingly, there have been many, many dev...",science,NaN
2,"Of course, nowadays electric cars are undoubte...",science,NaN
3,"Yet, it's worth talking about biofuels, as the...",science,NaN
4,"Knowing this, are biofuels a good option for o...",science,NaN
...,...,...,...
132370,"Immigrants, migrants, cross-country, cross-cul...",travel,NaN
132371,It also leverages global talent in service of us,travel,NaN
132372,all - look no further than the San Francisco B...,travel,NaN
132373,"It's not easy, but not impossible; in fact, no...",travel,NaN


In [ ]:
comb_med_2119_6_15_21_sentences

,sentence,topic,tenses
0,"It pollutes the environment, its limited in su...",science,NaN
1,"Unsurprisingly, there have been many, many dev...",science,NaN
2,"Of course, nowadays electric cars are undoubte...",science,NaN
3,"Yet, its worth talking about biofuels, as they...",science,NaN
4,"Knowing this, are biofuels a good option for o...",science,NaN
...,...,...,...
132611,"Immigrants, migrants, cross-country, cross-cul...",travel,NaN
132612,It also leverages global talent in service of us,travel,NaN
132613,all - look no further than the San Francisco B...,travel,NaN
132614,"Its not easy, but not impossible; in fact, not...",travel,NaN
